
Created on Fri Sep 18 14:29 2021 (Author: Clara Burgard)

This is an example script to apply multimelt



In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import cartopy
import cartopy.crs as ccrs
import matplotlib as mpl
import pandas as pd  
from tqdm.notebook import trange, tqdm
import time
import multimelt.melt_functions as meltf
import multimelt.plume_functions as pf
import multimelt.analysis_functions as ana
import multimelt.useful_functions as uf
from multimelt.constants import *
import seaborn as sns
import os, glob
import distributed

READ IN DATA

In [ ]:
inputpath_data= #path to folder containing file_other and file_conc
inputpath_mask = #path to folder containing file_isf_orig
inputpath_profiles = #path to folder containing file_TS_orig
outputpath_simple = #path to folder containing gammas_file_simple

inputpath_plumes =  #path to folder containing the file with plume characteristics
inputpath_boxes = #path to folder containing the file with box characteristics

outputpath_melt = #path to write out output

In [ ]:
# make the domain a little smaller to make the computation even more efficient - file isf has already been made smaller at its creation
map_lim = [-3000000,3000000]

In [ ]:
file_other = xr.open_dataset(inputpath_data+'corrected_draft_bathy_isf.nc') #file containing 'corrected_isfdraft'
file_other_cut = uf.cut_domain_stereo(file_other, map_lim, map_lim)
file_conc = xr.open_dataset(inputpath_data+'isfdraft_conc_Ant_stereo.nc') #file containing 'isfdraft_conc'
file_conc_cut = uf.cut_domain_stereo(file_conc, map_lim, map_lim)

In [ ]:
file_isf_orig = xr.open_dataset(inputpath_mask+'mask_file.nc') #file containing 'ISF_mask','latitude', 'longitude','front_bot_depth_avg', 'front_bot_depth_max','isf_name'
nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
large_isf = file_isf_nonnan['Nisf'].where(file_isf_nonnan['isf_area_here'] >= 2500, drop=True)
file_isf = file_isf_nonnan.sel(Nisf=large_isf)
file_isf['isf_name'] = file_isf['isf_name'].astype(str)
file_isf['region'] = file_isf['region'].astype(str)

file_TS_orig = xr.open_dataset(inputpath_profiles+'T_S_mean_prof_corrected_km_contshelf_and_offshore_1980-2018.nc') #file containing 'theta_ocean' and 'salinity_ocean'
file_TS = file_TS_orig.sel(Nisf=large_isf)

In [ ]:
box_charac_all_2D = xr.open_dataset(inputpath_boxes + 'nemo_5km_boxes_2D.nc')
box_charac_all_1D = xr.open_dataset(inputpath_boxes + 'nemo_5km_boxes_1D.nc')
plume_charac = xr.open_dataset(inputpath_plumes+'nemo_5km_plume_characteristics.nc')

PREPARE DATA

In [ ]:
lon = file_isf.longitude
lat = file_isf.latitude

In [ ]:
file_isf_conc = file_conc_cut['isfdraft_conc']

xx = file_isf.x
yy = file_isf.y
dx = (xx[2] - xx[1]).values
dy = (yy[2] - yy[1]).values
grid_cell_area = abs(dx*dy)  
grid_cell_area_weighted = file_isf_conc * grid_cell_area

In [ ]:
ice_draft_pos = file_other_cut['corrected_isfdraft']
ice_draft_neg = -ice_draft_pos

In [ ]:
isf_stack_mask = uf.create_stacked_mask(file_isf['ISF_mask'], file_isf.Nisf, ['y','x'], 'mask_coord')

In [ ]:
param_var_of_int_2D = file_isf[['ISF_mask', 'latitude', 'longitude', 'dGL']]
param_var_of_int_1D = file_isf[['front_bot_depth_avg', 'front_bot_depth_max','isf_name']]

In [ ]:
geometry_info_2D = plume_charac.merge(param_var_of_int_2D).merge(ice_draft_pos).rename({'corrected_isfdraft':'ice_draft_pos'}).merge(grid_cell_area_weighted).rename({'isfdraft_conc':'grid_cell_area_weighted'}).merge(file_isf_conc)
geometry_info_1D = param_var_of_int_1D

SIMPLE PARAMS

In [ ]:
nisf_list = geometry_info_1D.Nisf
T_S_profile = file_TS.ffill(dim='depth')

mparam = # POSSIBILITIES: ['linear_local', 'quadratic_local', 'quadratic_local_locslope', 'quadratic_local_cavslope', 'quadratic_mixed_mean', 'quadratic_mixed_locslope','quadratic_mixed_cavslope'] 

gamma = # fill in
ds_2D, ds_1D = meltf.calculate_melt_rate_1D_and_2D_all_isf(nisf_list, 
                                                           T_S_profile, 
                                                           geometry_info_2D, 
                                                           geometry_info_1D, 
                                                           isf_stack_mask, 
                                                           mparam, 
                                                           gamma, 
                                                           U_param=True)


ds_2D.to_netcdf(outputpath_melt+'melt_rates_2D_'+mparam+'.nc')
ds_1D.to_netcdf(outputpath_melt+'melt_rates_1D_'+mparam+'.nc')

PLUMES

In [ ]:
nisf_list = geometry_info_1D.Nisf
T_S_profile = file_TS.ffill(dim='depth')

mparam = # POSSIBILITIES: ['lazero19', 'lazero19_modif']

gamma = # fill in
E0 = # fill in
                 
ds_2D, ds_1D = meltf.calculate_melt_rate_1D_and_2D_all_isf(nisf_list, 
                                                           T_S_profile, 
                                                           geometry_info_2D, 
                                                           geometry_info_1D, 
                                                           isf_stack_mask, 
                                                           mparam, 
                                                           gamma, 
                                                           E0=E0, 
                                                           verbose=True)
            
ds_2D.to_netcdf(outputpath_melt+'melt_rates_2D_'+mparam+'.nc')
ds_1D.to_netcdf(outputpath_melt+'melt_rates_1D_'+mparam+'.nc')

BOXES

In [ ]:
nisf_list = geometry_info_1D.Nisf
T_S_profile = file_TS.ffill(dim='depth') 
picop_opt = 'no'

nD_config = # POSSIBILITIES: 1 to 4
pism_version = # POSSIBILITIES: 'yes' or 'no'

mparam = 'boxes_'+str(nD_config)+'_pism'+pism_version+'_picop'+picop_opt

C = # fill in
gamma = # fill in

ds_2D, ds_1D = meltf.calculate_melt_rate_1D_and_2D_all_isf(nisf_list, 
                                                           T_S_profile, 
                                                           geometry_info_2D, 
                                                           geometry_info_1D, 
                                                           isf_stack_mask, 
                                                           mparam, 
                                                           gamma,
                                                           C=C, 
                                                           angle_option='local', 
                                                           box_charac_2D=box_charac_all_2D, 
                                                           box_charac_1D=box_charac_all_1D, 
                                                           box_tot=nD_config, 
                                                           box_tot_option='nD_config', 
                                                           pism_version=pism_version, 
                                                           picop_opt=picop_opt)
                                       
ds_2D.to_netcdf(outputpath_melt+'melt_rates_2D_'+mparam+'.nc')
ds_1D.to_netcdf(outputpath_melt+'melt_rates_1D_'+mparam+'.nc')

PICOP

In [ ]:
nisf_list = geometry_info_1D.Nisf
T_S_profile = file_TS.ffill(dim='depth') 

nD_config = # POSSIBILITIES: 1 to 4    
pism_version = # POSSIBILITIES: 'yes' or 'no'
picop_opt = # POSSIBILITIES: '2018' or '2019'

mparam = 'boxes_'+str(nD_config)+'_pism'+pism_version+'_picopyes'

C = # for box part - fill in
gamma = # for box part - fill in

gamma_plume = # for plume part - fill in
E0 = # for plume part - fill in

ds_2D, ds_1D = meltf.calculate_melt_rate_1D_and_2D_all_isf(nisf_list, 
                                                           T_S_profile, 
                                                           geometry_info_2D, 
                                                           geometry_info_1D, 
                                                           isf_stack_mask, 
                                                           mparam, 
                                                           gamma,
                                                           C=C,
                                                           E0=E0, 
                                                           angle_option='local',
                                                           box_charac_2D=box_charac_all_2D, 
                                                           box_charac_1D=box_charac_all_1D, 
                                                           box_tot=nD_config, 
                                                           box_tot_option='nD_config', 
                                                           pism_version=pism_version,
                                                           picop_opt=picop_opt, 
                                                           gamma_plume=gamma_plume)

ds_2D.to_netcdf(outputpath_melt+'melt_rates_2D_'+mparam+'.nc')
ds_1D.to_netcdf(outputpath_melt+'melt_rates_1D_'+mparam+'.nc')